In [1]:
# !export TOKENIZERS_PARALLELISM=false
# !python -m pip install --upgrade pip
# !pip install typing_extensions pydantic openai
# !pip install datasets transformers peft trl bitsandbytes
# !pip install sentence-transformers langchain langchain_community
# !pip install chromadb
# #!pip uninstall numpy -y
# # !pip install numpy==1.26.4
# !pip install "numpy<2"

In [ ]:
from huggingface_hub import login
login(token='')

from peft import PeftModel

from transformers import AutoModelForCausalLM

model_name = "google/gemma-3-4b-it"

base_model = AutoModelForCausalLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, "jeongyoonhuh/q_lora_korqa")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: Loading a multimodal model with `AutoModelForCausalLM` is deprecated and will be removed in v5. `AutoModelForCausalLM` will be used to load only the text-to-text generation module.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from langchain.prompts import PromptTemplate
from langchain.callbacks.tracers import LangChainTracer
from langchain.callbacks.manager import CallbackManager
from langsmith import Client
import os
import uuid
import time
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import Chroma
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Pipeline 래핑
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.1,
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# 임베딩 및 벡터스토어 로드
embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
vector_store = Chroma(
    persist_directory="chroma_index",
    embedding_function=embedding
)
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.7
    }
)

# ✅ 사용자 정의 프롬프트
custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
        너는 항공기 반입 금지 및 제한 물품에 대하여 친절하게 답해주고 나라별 여행지 추천 및 문화차이를 잘 설명할 수 있어
        문맥에 해당 하는 내용들만 신뢰도있게 대답해야해

        ### 질문:
        {question}
        
        ### 문맥:
        {context}

        ### 답변:
    """
)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJa

In [4]:
os.environ["LANGCHAIN_API_KEY"] = "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"  # 실제 API 키로 변경
os.environ["LANGCHAIN_PROJECT"] = "travel_sj_project"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

client = Client()

# 메모리 객체 생성
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 대화형 QA 체인 생성
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": custom_prompt}
)

/tmp/ipykernel_4865/67609685.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [5]:
# ✅ 국가 목록 상수 추가
SUPPORTED_COUNTRIES = [
    "한국", "일본", "베트남", "미국", "필리핀", "싱가포르", "대만",
    "인도네시아", "홍콩", "중국", "말레이시아"
]

def detect_country(query):
    """사용자 질문에서 국가 감지"""
    query_lower = query.lower()

    # ✅ 기존 조건 → 딕셔너리로 정리
    country_map = {
        "일본": ["일본", "재팬", "도쿄", "교토", "오사카"],
        "대만": ["대만", "타이완", "타이페이"],
        "한국": ["한국", "서울", "부산", "제주"],
        "베트남": ["베트남", "하노이", "호치민", "다낭", "하롱베이"],
        "미국": ["미국", "뉴욕", "로스앤젤레스", "시카고", "라스베가스", "하와이"],
        "필리핀": ["필리핀", "마닐라", "세부", "보라카이", "팔라완"],
        "싱가포르": ["싱가포르", "센토사", "마리나베이", "가든스바이더베이"],
        "인도네시아": ["인도네시아", "발리", "자카르타", "롬복"],
        "홍콩": ["홍콩", "침사추이", "란타우", "빅토리아피크"],
        "중국": ["중국", "베이징", "상하이", "광저우", "시안", "만리장성"],
        "말레이시아": ["말레이시아", "쿠알라룸푸르", "페낭", "코타키나발루", "랑카위"]
    }

    for country, keywords in country_map.items():
        if any(keyword in query_lower for keyword in keywords):
            return country

    return "Unknown"  # ✅ 감지되지 않으면 Unknown 반환


def answer_question(query, qa_chain, vector_store, chat_history=None):
    """국가 필터링을 적용한 질문 처리"""

    # 국가 감지
    country = detect_country(query)

    # ✅ 없는 나라 처리 추가
    if country == "Unknown":
        return {
            "result": "관련 데이터가 없습니다.",
            "source_documents": [],
            "question": query
        }

    try:
        is_conv_chain = hasattr(qa_chain, 'memory')

        country_keywords = {
            '일본': ['일본', '재팬', '도쿄'],
            '대만': ['대만', '타이완', '타이페이'],
            '한국': ['한국', '서울', '부산'],
            '베트남': ['베트남', '하노이', '호치민'],
            '미국': ['미국', '뉴욕', '워싱턴'],
            '필리핀': ['필리핀', '마닐라', '세부'],
            '싱가포르': ['싱가포르', '센토사'],
            '인도네시아': ['인도네시아', '발리', '자카르타'],
            '홍콩': ['홍콩'],
            '중국': ['중국', '베이징', '상하이'],
            '말레이시아': ['말레이시아', '쿠알라룸푸르']
        }

        enhanced_query = f"{country_keywords.get(country, [country])[0]} {query}"

        if is_conv_chain:
            if chat_history is not None:
                result = qa_chain({"question": enhanced_query, "chat_history": chat_history})
            else:
                result = qa_chain({"question": enhanced_query})
        else:
            retriever = vector_store.as_retriever(search_kwargs={"k": 5})
            docs = retriever.get_relevant_documents(enhanced_query)

            keywords = country_keywords.get(country, [country.lower()])
            filtered_docs = [
                doc for doc in docs
                if any(keyword.lower() in doc.metadata.get('source', '').lower() or
                       keyword.lower() in doc.page_content.lower()
                       for keyword in keywords)
            ]

            if not filtered_docs:
                filtered_docs = docs

            result = qa_chain({
                "input_documents": filtered_docs,
                "query": query
            })

        return result
    except Exception as e:
        print(f"처리 중 오류 발생: {e}")
        return {
            "result": f"처리 중 오류가 발생했습니다: {str(e)}",
            "source_documents": [],
            "question": query
        }

In [11]:
# 첫 번째 질문
query = "대만에 소세지 들고가도돼??"
result = answer_question(query, qa_chain, vector_store)
print("Answer:", result)

# 후속 질문
query = "그럼 육포는 어때?"
result = answer_question(query, qa_chain, vector_store)
print("Answer:", result.get('result', '응답 없음'))

Answer: {'question': '대만 대만에 소세지 들고가도돼??', 'chat_history': [HumanMessage(content='대만 대만에 소세지 반입 돼??', additional_kwargs={}, response_metadata={}), AIMessage(content='\n        너는 항공기 반입 금지 및 제한 물품에 대하여 친절하게 답해주고 나라별 여행지 추천 및 문화차이를 잘 설명할 수 있어\n        문맥에 해당 하는 내용들만 신뢰도있게 대답해야해\n\n        ### 질문:\n        대만 대만에 소세지 반입 돼??\n        \n        ### 문맥:\n        바로 기차를 제외한 대중교통(버스, 지하철)에서 음식을 섭취하는 행동입니다. 특히 여러분들이 대만 여행을 할 때 가장 많이 이용하는 교통수단인 지하철MRT에서 주의하셔야 하는데요 지하철 역사에서는 취식이 가능하나 교통카드를 찍고 플랫폼 안으로 들어가는 순간 취식이 금지되니 꼭!!!! 기억해주세요 위반시 최대 10,000NTD (한화 38만원)의 벌금이 부과됩니다. 버스에서 거스름돈을 주지 않는다 한국과는 다르게 대만의 버스는 거스름돈을 주지 않습니다. 버스를 탑승하기 전 반드시 동전을 확인하셔야겠죠? 좁은 길에서는 이렇게 말하세요 여러분들이 대만 여행을 할 때 꼭 알고 있어야 할 중요한 중국어 입니다. 한국에서도 좁은 길을 비집고 들어오거나 밀치는 것은 상당히 무례한 행동이죠?? 대만 사람들도 이러한 행위들을 정말 불쾌하게 생각하고 있습니다. 그렇지만 이렇게 얘기한다면 자비로운 표정으로 길을 비켜줄 거예요 바로 찌에꿔이샤 입니다. 갑자기 길을 비켜준다고 협박하는 건 아닐까? 의심하지 마세요 한국말로 정중하게 지나가겠습니다 라는 표현입니다. 표현이 어렵다면 더욱 쉬운 표현인 뿌하오이쓰 라고 얘기하세요 이렇게 얘기하면 길을 비켜주는 사람들도 내가 길을 막아서 미안하다는 뜻으로 뿌하오이쓰 라고 대답하며 웃으며 길을 비켜 준답니다. 뿌

In [7]:
#result = result['chat_history'][1].content.split("### 답변:")[1]

In [8]:
# query = "아프리카에 반입금지 물품이 뭐야??"
# result = answer_question(query, qa_chain, vector_store)
# print("Answer:", result.get('result', '응답 없음'))  # 🔥 관련 데이터가 없습니다. 출력됨

# query = "아프리카에 지켜야 할 에티켓이 뭐야??"
# result = answer_question(query, qa_chain, vector_store)
# print("Answer:", result.get('result', '응답 없음'))  # 🔥 관련 데이터가 없습니다. 출력됨

In [9]:
#!pip install gradio

In [10]:
import gradio as gr

def gradio_answer_question(query):
    result = answer_question(query, qa_chain, vector_store)
    output = result['chat_history'][1].content.split("### 답변:")[1]
    return output

iface = gr.Interface(
    fn=gradio_answer_question,
    inputs=gr.Textbox(lines=2, placeholder="여행 관련 질문을 입력하세요."),
    outputs=gr.Textbox(),
    title="여행 정보 챗봇",
    description="국가별 반입 금지 품목 및 문화 차이에 대해 질문하세요."
)

iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://626155aacba2ec676c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
